In [62]:
import pandas as pd
import numpy as np

In [9]:
data_2016 = pd.read_csv('../../data/Medical_Claims_Interns_Update_2016.csv')
data_2017 = pd.read_csv('../../data/Medical_Claims_Interns_Update_2017.csv')
data_2018 = pd.read_csv('../../data/Medical_Claims_Interns_Update_2018.csv')

In [12]:
med_data = pd.concat([data_2016, data_2017, data_2018], ignore_index=True)
med_data.head()

,Member ID Encrypted,Claim ID,Service Line,Member Gender,Age,Zip (5-digit),MSA,County,State,Payer Type,...,Bill Type Class,Billed,Total Billed,Allowed,Total Allowed,Paid,Total Paid,HCG Setting,HCG Detail,Incurred Year
0,710087KT90EY0DZUV,16365C1CB1000,1,F,46,98335,Tacoma-Lakewood WA,PIERCE,WA - Washington,HMO - Commercial HMO,...,Unknown,$358.00,$358.00,$341.89,$341.89,$341.89,$341.89,3. Professional,P43 - PHY Preventive Physical Exams,2016
1,710087KT90EY0DZUV,16365C1CB1000,1,F,46,98335,Tacoma-Lakewood WA,PIERCE,WA - Washington,HMO - Commercial HMO,...,Unknown,-$358.00,-$358.00,-$341.89,-$341.89,-$341.89,-$341.89,3. Professional,P43 - PHY Preventive Physical Exams,2016
2,710087KT90EY0DZUV,16365C1CB1000,2,F,46,98335,Tacoma-Lakewood WA,PIERCE,WA - Washington,HMO - Commercial HMO,...,Unknown,$252.00,$252.00,$240.66,$240.66,$240.66,$240.66,3. Professional,P40a - PHY Preventive Other - General,2016
3,710087KT90EY0DZUV,16365C1CB1000,2,F,46,98335,Tacoma-Lakewood WA,PIERCE,WA - Washington,HMO - Commercial HMO,...,Unknown,-$252.00,-$252.00,-$240.66,-$240.66,-$240.66,-$240.66,3. Professional,P40a - PHY Preventive Other - General,2016
4,710087KT90EY0DZUV,16365C1CB1001,1,F,46,98335,Tacoma-Lakewood WA,PIERCE,WA - Washington,HMO - Commercial HMO,...,Unknown,$358.00,$358.00,$341.89,$341.89,$341.89,$341.89,3. Professional,P43 - PHY Preventive Physical Exams,2016


In [13]:
zip_class = pd.read_csv('../../data/final310.csv')

In [14]:
med_data = med_data.loc[med_data['Relation'] == 'SUBSCRIBER']

In [15]:
# Dict: county -> urban/rural
# urban_rural = {'ADAMS': 'rural', 'ASOTIN': 'rural', 'BENTON': 'urban', 'CHELAN': 'rural', 'CLALLAM': 'rural', 'CLARK': 'urban', 'COLUMBIA': 'rural', 'COWLITZ': 'rural', 'DOUGLAS': 'rural', 'FERRY': 'rural', 'FRANKLIN': 'rural', 'GARFIELD': 'rural', 'GRANT': 'rural', 'GRAYS HARBOR': 'rural', 'ISLAND': 'rural', 'JEFFERSON': 'rural', 'KING': 'urban', 'KITSAP': 'urban', 'KITTITAS': 'rural', 'KLICKITAT': 'rural', 'LEWIS': 'rural', 'LINCOLN': 'rural', 'MASON': 'rural', 'OKANOGAN': 'rural', 'PACIFIC': 'rural', 'PEND OREILLE': 'rural', 'PIERCE': 'urban', 'SAN JUAN': 'rural', 'SKAGIT': 'rural', 'SKAMANIA': 'rural', 'SNOHOMISH': 'urban', 'SPOKANE': 'urban', 'STEVENS': 'rural', 'THURSTON': 'urban', 'WAHKIAKUM': 'rural', 'WALLA WALLA': 'rural', 'WHATCOM': 'rural', 'WHITMAN': 'rural', 'YAKIMA': 'rural'}

In [69]:
def reshape(group):
    df = pd.DataFrame()

    # Demographic info
    df['member_id'] = group.head(1)['Member ID Encrypted']
    df['year'] = group.head(1)['Incurred Year']
    df['gender'] = group.head(1)['Member Gender']
    df['age'] = group.head(1)['Age']
    zipcode = group.head(1)['Zip (5-digit)'].iloc[0]
    df['postal_code'] = zipcode
    county = group.head(1)['County'].iloc[0]
    df['county'] = county
#     if county in urban_rural:
#         df['urban_rural'] = urban_rural[county]
    if zipcode in zip_class['ZIPCODEN'].values:
        df['urban_rural'] = zip_class.loc[zip_class['ZIPCODEN'] == zipcode]['RUCA30'].iloc[0]
    
    # Get all ICD codes and Rollups for this group
    icd_codes = group['Primary ICD Diagnosis Code'] + group['2nd ICD Diagnosis Code'] + group['3rd ICD Diagnosis Code'] + group['4th ICD Diagnosis Code'] + group['5th ICD Diagnosis Code'] + group['6th ICD Diagnosis Code'] + group['7th ICD Diagnosis Code'] + group['8th ICD Diagnosis Code'] + group['9th ICD Diagnosis Code'] + group['10th ICD Diagnosis Code']
    icd_rollups = group['Primary ICD Rollup'] + group['2nd ICD Rollup'] + group['3rd ICD Rollup'] + group['4th ICD Rollup'] + group['5th ICD Rollup'] + group['6th ICD Rollup'] + group['7th ICD Rollup'] + group['8th ICD Rollup'] + group['9th ICD Rollup'] + group['10th ICD Rollup']
    
    # Set binary for conditions: 1 if condition exists, 0 otherwise
    df['binary_asthma'] = 1 if any('Asthma' in x for x in icd_rollups) else 0
    df['binary_bp'] = 1 if any('I10' in x for x in icd_codes) else 0
    df['binary_cancer'] = 1 if any('Cancer' in x for x in icd_rollups) else 0
    df['binary_cardiovascular'] = 1 if any('heart' in x for x in icd_rollups) else 0
    df['binary_copd'] = 1 if any('Chronic obstructive pulmonary disease' in x for x in icd_rollups) else 0
    df['binary_diabetes'] = 1 if any('Diabetes' in x for x in icd_rollups) else 0
    df['binary_kidney'] = 1 if any('N18' in x for x in icd_codes) else 0
    df['binary_obesity'] = 1 if any('E66' in x for x in icd_codes) else 0
    df['binary_musculoskeletal'] = 1 if any('musculoskeletal' in s for s in icd_rollups) else 0
    df['binary_cholesterol'] = 1 if any('E78' in x for x in icd_codes) else 0
    #doesn't contain Bipolar disorders, Eating disorders, Psychogenic disorders, Other miscellaneous mental conditions, 
                                    #Codes related to mental health disorders, Dissociative diorders, Somatoform disorders
    mental_health = ['Anxiety disorders', 'Adjustment disorders', 'Mood disorders', 
                     'Schizophrenia and other psychotic disorders', 
                     'Suicide and intentional self-inflicted injury', 
                     'Impulse control disorders not elsewhere classified', 
                     'Screening and history of mental health and substance abuse codes', 
                    'Miscellaneous mental disorders']
    df['binary_mental_disorder'] = 1 if any(x in mental_health for x in icd_rollups) or any('G4700' in x for x in icd_codes) else 0
    
    # Initialize count columns
    df['claims_asthma'] = 0
    df['claims_bp'] = 0
    df['claims_cancer'] = 0
    df['claims_cardiovascular'] = 0
    df['claims_copd'] = 0
    df['claims_diabetes'] = 0
    df['claims_kidney'] = 0
    df['claims_obesity'] = 0
    df['claims_musculoskeletal'] = 0
    df['claims_cholesterol'] = 0
    df['claims_mental_disorder'] = 0
    df['total_claims'] = 0
    
    # Group by Claim ID to get unique claim counts
    claim_grouped = group.groupby(['Claim ID'], as_index=False)
    for claim_id, claim_group in claim_grouped:
        claim_reshape(claim_group, df)

    # Total number of conditions
    total_conditions = df['binary_kidney'] + df['binary_mental_disorder'] + df['binary_bp'] + df['binary_cardiovascular'] + df['binary_diabetes'] + df['binary_obesity'] + df['binary_cancer'] + df['binary_musculoskeletal'] + df['binary_copd'] + df['binary_cholesterol']
    df['total_conditions'] = total_conditions
    
    # Flags for nonused (claims were not filed for our conditions) and comorbidity (has more than one condition out of the ones we are interested in)
    df['flag_nonused'] = df['total_claims'].apply(lambda x: 1 if x == 0 else 0)
    df['flag_comorbidity'] = total_conditions.apply(lambda x: 1 if x > 1 else 0)

    return df

In [70]:
def claim_reshape(claim_group, df): 
    # Lists of all codes and rollups
    icd_codes = claim_group['Primary ICD Diagnosis Code'] + claim_group['2nd ICD Diagnosis Code'] + claim_group['3rd ICD Diagnosis Code'] + claim_group['4th ICD Diagnosis Code'] + claim_group['5th ICD Diagnosis Code'] + claim_group['6th ICD Diagnosis Code'] + claim_group['7th ICD Diagnosis Code'] + claim_group['8th ICD Diagnosis Code'] + claim_group['9th ICD Diagnosis Code'] + claim_group['10th ICD Diagnosis Code']
    icd_rollups = claim_group['Primary ICD Rollup'] + claim_group['2nd ICD Rollup'] + claim_group['3rd ICD Rollup'] + claim_group['4th ICD Rollup'] + claim_group['5th ICD Rollup'] + claim_group['6th ICD Rollup'] + claim_group['7th ICD Rollup'] + claim_group['8th ICD Rollup'] + claim_group['9th ICD Rollup'] + claim_group['10th ICD Rollup']
       
    if any('Asthma' in x for x in icd_rollups):
        df['claims_asthma'] += 1  
    if any('I10' in x for x in icd_codes):
        df['claims_bp'] += 1
    if any('Cancer' in x for x in icd_rollups):
        df['claims_cancer'] += 1
    if any('heart' in x for x in icd_rollups):
        df['claims_cardiovascular'] += 1
    if any('Chronic obstructive pulmonary disease' in x for x in icd_rollups):
        df['claims_copd'] += 1
    if any('Diabetes' in x for x in icd_rollups):
        df['claims_diabetes'] += 1
    if any('N18' in x for x in icd_codes):
        df['claims_kidney'] += 1
    if any('E66' in x for x in icd_codes):
        df['claims_obesity'] += 1
    if any('E78' in x for x in icd_codes):
        df['claims_cholesterol'] += 1
    if any('musculoskeletal' in x for x in icd_rollups):
        df['claims_musculoskeletal'] += 1
    mental_health = ['Anxiety disorders', 'Adjustment disorders', 'Mood disorders', 
                     'Schizophrenia and other psychotic disorders', 
                     'Suicide and intentional self-inflicted injury', 
                     'Impulse control disorders not elsewhere classified', 
                     'Screening and history of mental health and substance abuse codes', 
                    'Miscellaneous mental disorders']
    if any(x in mental_health for x in icd_rollups) or any('G4700' in x for x in icd_codes):
        df['claims_mental_disorder'] += 1

    df['total_claims'] += 1

In [71]:
# Reshape data by grouping by member and year
reshaped_med_data = med_data.groupby(['Member ID Encrypted', 'Incurred Year'], as_index=False).apply(reshape)

In [72]:
columns = ['member_id', 'year', 'age', 'gender', 'postal_code', 'county', 'urban_rural', 'binary_asthma', 'binary_bp', 'binary_cancer', 'binary_cardiovascular', 'binary_cholesterol', 'binary_copd', 'binary_diabetes', 'binary_kidney', 'binary_mental_disorder', 'binary_musculoskeletal', 'binary_obesity', 'claims_asthma', 'claims_bp', 'claims_cancer', 'claims_cardiovascular', 'claims_cholesterol', 'claims_copd', 'claims_diabetes', 'claims_kidney', 'claims_mental_disorder', 'claims_musculoskeletal', 'claims_obesity', 'flag_comorbidity', 'flag_nonused', 'total_claims', 'total_conditions']

In [73]:
reshaped_med_data = reshaped_med_data[columns]

In [77]:
reshaped_med_data.to_csv('../../data/reshaped_med.csv', index=False)

In [74]:
reshaped_med_data.iloc[:,0:16]

,,member_id,year,age,gender,postal_code,county,urban_rural,binary_asthma,binary_bp,binary_cancer,binary_cardiovascular,binary_cholesterol,binary_copd,binary_diabetes,binary_kidney,binary_mental_disorder
0,614282,71002LEM4JV1PJFM4,2017,34,F,98030,KING,1.0,0,0,1,0,0,0,0,0,0
1,0,710087KT90EY0DZUV,2016,46,F,98335,PIERCE,1.0,0,0,0,0,0,0,0,0,0
2,614311,710087KT90EY0DZUV,2017,46,F,98335,PIERCE,1.0,0,0,0,0,0,0,0,0,0
3,1233380,710087KT90EY0DZUV,2018,47,F,98335,PIERCE,1.0,1,0,0,1,0,1,0,0,0
4,8,71008ADZ2VBBDM14C,2016,53,F,98032,KING,1.0,0,0,0,0,0,0,1,0,0
5,614330,71008ADZ2VBBDM14C,2017,54,F,98032,KING,1.0,0,0,0,0,1,0,1,0,0
6,1233419,71008ADZ2VBBDM14C,2018,55,F,98032,KING,1.0,0,0,0,0,1,0,1,0,0
7,14,7100UKNUMFS22SCRI,2016,46,M,99156,PEND OREILLE,2.0,0,0,0,0,0,0,0,0,0
8,614346,7100UKNUMFS22SCRI,2017,47,M,99156,PEND OREILLE,2.0,0,0,0,0,0,1,0,0,0
9,36,7100UNBYH5SYEX19X,2016,62,F,98926,KITTITAS,4.0,0,0,0,0,0,0,0,0,0


In [78]:
pd.read_csv('../../data/reshaped_med.csv')

,member_id,year,age,gender,postal_code,county,urban_rural,binary_asthma,binary_bp,binary_cancer,...,claims_copd,claims_diabetes,claims_kidney,claims_mental_disorder,claims_musculoskeletal,claims_obesity,flag_comorbidity,flag_nonused,total_claims,total_conditions
0,71002LEM4JV1PJFM4,2017,34,F,98030,KING,1.0,0,0,1,...,0,0,0,0,0,0,0,0,3,1
1,710087KT90EY0DZUV,2016,46,F,98335,PIERCE,1.0,0,0,0,...,0,0,0,0,0,2,0,0,3,1
2,710087KT90EY0DZUV,2017,46,F,98335,PIERCE,1.0,0,0,0,...,0,0,0,0,0,8,0,0,10,1
3,710087KT90EY0DZUV,2018,47,F,98335,PIERCE,1.0,1,0,0,...,1,0,0,0,0,15,1,0,24,3
4,71008ADZ2VBBDM14C,2016,53,F,98032,KING,1.0,0,0,0,...,0,3,0,0,0,0,0,0,5,1
5,71008ADZ2VBBDM14C,2017,54,F,98032,KING,1.0,0,0,0,...,0,8,0,0,0,1,1,0,13,3
6,71008ADZ2VBBDM14C,2018,55,F,98032,KING,1.0,0,0,0,...,0,7,0,0,0,1,1,0,11,3
7,7100UKNUMFS22SCRI,2016,46,M,99156,PEND OREILLE,2.0,0,0,0,...,0,0,0,0,0,0,0,0,6,0
8,7100UKNUMFS22SCRI,2017,47,M,99156,PEND OREILLE,2.0,0,0,0,...,2,0,0,0,0,0,0,0,6,1
9,7100UNBYH5SYEX19X,2016,62,F,98926,KITTITAS,4.0,0,0,0,...,0,0,0,0,11,0,0,0,56,1


In [ ]:
# Tests if the number of claims for both reshaped and original(actual) are the same
def test(memberid, year):
    reshaped_claims = reshaped_med_data.loc[(reshaped_med_data['member_id'] == memberid) & 
                      (reshaped_med_data['year'] == year),].total_claims.item()
    print('Reshaped number of claims: ', reshaped_claims)
    
    actual = med.loc[(med['Member ID Encrypted'] == memberid) & (med['Incurred Year'] == year),]
    print('Actual number of claims: ', len(actual['Claim ID'].unique()))
    
    print(reshaped_claims == len(actual['Claim ID'].unique()))

In [ ]:
test('71069K607Y1CQ9BV5', 2016)

In [ ]:
test('71069K607Y1CQ9BV5', 2017)

In [ ]:
test('71069K607Y1CQ9BV5', 2018)